In [1]:
## import json file
import json
import pandas as pd
import numpy as np

# import both weekly and year files
with open('NFL_data_weekly.json') as json_data:
    data = json.load(json_data)
with open('NFL_data_yearly.json') as json_data:
    data_yearly = json.load(json_data)

In [2]:
## convert the list into dictionary form
d = {}
for i in range(len(data)):
    # key := ('Position', 'year', 'week') format
    key = tuple(list(data[i].keys())[0].split('-'))
    d[key] = list(data[i].values())[0]

d_yearly = {}
for i in range(len(data_yearly)):
    key = tuple(list(data_yearly[i].keys())[0].split('-'))
    d_yearly[key] = list(data_yearly[i].values())[0]
    

In [3]:
key = ('QB', '2011', '4')
d[key][1]


['GB',
 'PHI',
 'CAR',
 'SEA',
 'DAL',
 'TB',
 'TEN',
 'NYG',
 'SF',
 'IND',
 'DEN',
 'NE',
 'CIN',
 'DET',
 'ATL',
 'SD',
 'CLE',
 'MIN',
 'KC',
 'OAK',
 'NO',
 'JAX',
 'HOU',
 'STL',
 'PIT',
 'WAS',
 'BUF',
 'ARI',
 'MIA',
 'BAL',
 'CHI',
 'DEN',
 'TEN',
 'TB',
 'NYJ',
 'MIA',
 'GB']

In [6]:
for pos in ['QB', 'RB', 'WR', 'TE']:
    key = (pos, '2011', '4')
    titles = d[key][3]
    players = d[key][0]
    stats = d[key][4]

    # make a pandas dataframe
    df2 = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'team', 'Fantasy Points']
                      + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                      + [t + '_prev_1yr' for t in titles])
    
    for year in range(2011, 2018):
        for week in range(4, 18):
            key = (pos, str(year), str(week))
            # get statistics types, name of players, and statistics
            titles = d[key][3]
            players = d[key][0]
            stats = d[key][4]

            # make a new dataframe 
            df = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'team', 'Fantasy Points']
                              + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                              + [t + '_prev_1yr' for t in titles])

            df['player'] = players
            df['pos'] = list(key)[0]
            df['year'] = list(key)[1]
            df['week'] = list(key)[2]
            df['team'] = d[key][1]
            df['Fantasy Points'] = stats[0]

            ## extract data of the previous weeks
            # keys for previous 3 weeks
            key_prevs = [tuple([k for k in list(key)[:-1]] + [str(int(key[2])-i)]) for i in range(1, 4)]

            # add data of previous weeks respectively
            for prev in range(len(key_prevs)):
                # extract key
                key_prev = key_prevs[prev]
                players_prev = d[key_prev][0]
                stats_prev = d[key_prev][4]

                # add all information of the previous week
                for idx_prev in range(len(players_prev)):
                    # check if the player exists
                    if players_prev[idx_prev] in players:
                        idx = players.index(players_prev[idx_prev])
                    else:
                        continue
                    for idx_col in range(len(stats_prev)):
                        df.iloc[idx, idx_col+6+len(titles)*prev] = stats_prev[idx_col][idx_prev]

            ## extract data of the previous year
            key_yearly = (pos, str(year-1), 'all')
            players_prev_yearly = d_yearly[key_yearly][0]
            stats_prev_yearly = d_yearly[key_yearly][2][1:]

            # add all information of the previous year
            for idx_prev in range(len(players_prev_yearly)):
                # check if the player exists
                if players_prev_yearly[idx_prev] in players:
                    idx = players.index(players_prev_yearly[idx_prev])
                else:
                    continue

                for idx_col in range(len(stats_prev_yearly)):
                    df.iloc[idx, idx_col+6+len(titles)*len(key_prevs)] = stats_prev_yearly[idx_col][idx_prev]

            df2 = df2.append(df)
            
    for title in titles:
        temp_1wk = title + '_prev'
        temp_2wk = title + '_prev_2wk'
        temp_3wk = title + '_prev_3wk'

        df_temp = df2[[temp_1wk, temp_2wk, temp_3wk]]
        df_temp = df_temp.astype(float)
        df2[title+'_momentum'] = df_temp.apply(lambda r: np.nan if r.isnull().sum() >= 2 else r.mean(), axis=1)
        
    df3 = df2[['pos', 'year', 'week', 'player', 'team', 'Fantasy Points'] + [t + '_prev' for t in titles] 
              + [t + '_momentum' for t in titles] + [t + '_prev_1yr' for t in titles]]
    # remove ',' from numbers (e.g.: 1,523 -> 1523)
    df3['Passing Yards_prev_1yr'] = pd.Series(df3['Passing Yards_prev_1yr']).str.replace(',', '');
    
    df3.to_csv(pos+'_adj.csv', index=False)

/Users/younghojung/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


KeyboardInterrupt: 

In [7]:
pos = 'QB'
key = (pos, '2011', '4')


titles = d[key][3]
players = d[key][0]
stats = d[key][4]

# make a new dataframe 
df = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'team', 'Fantasy Points']
                  + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                  + [t + '_prev_1yr' for t in titles])

df['player'] = players
df['pos'] = list(key)[0]
df['year'] = list(key)[1]
df['week'] = list(key)[2]
df['team'] = d[key][1]
df['Fantasy Points'] = stats[0]

## extract data of the previous weeks
# keys for previous 3 weeks
key_prevs = [tuple([k for k in list(key)[:-1]] + [str(int(key[2])-i)]) for i in range(1, 4)]

# add data of previous weeks respectively
for prev in range(len(key_prevs)):
    # extract key
    key_prev = key_prevs[prev]
    players_prev = d[key_prev][0]
    stats_prev = d[key_prev][4]

    # add all information of the previous week
    for idx_prev in range(len(players_prev)):
        # check if the player exists
        if players_prev[idx_prev] in players:
            idx = players.index(players_prev[idx_prev])
        else:
            continue
        for idx_col in range(len(stats_prev)):
            df.iloc[idx, idx_col+6+len(titles)*prev] = stats_prev[idx_col][idx_prev]

## extract data of the previous year
key_yearly = (pos, str(year-1), 'all')
players_prev_yearly = d_yearly[key_yearly][0]
stats_prev_yearly = d_yearly[key_yearly][2][1:]

# add all information of the previous year
for idx_prev in range(len(players_prev_yearly)):
    # check if the player exists
    if players_prev_yearly[idx_prev] in players:
        idx = players.index(players_prev_yearly[idx_prev])
    else:
        continue

    for idx_col in range(len(stats_prev_yearly)):
        df.iloc[idx, idx_col+6+len(titles)*len(key_prevs)] = stats_prev_yearly[idx_col][idx_prev]

df2 = df2.append(df)


In [12]:
temp = df[['pos', 'year', 'week', 'player', 'Pass Attempts_prev', 'Pass Attempts_prev_2wk', 'Pass Attempts_prev_3wk']].head(5)



,pos,year,week,player,Pass Attempts_prev,Pass Attempts_prev_2wk,Pass Attempts_prev_3wk
0,QB,2011,4,Aaron Rodgers,38,30,35
1,QB,2011,4,Michael Vick,23,28,32
2,QB,2011,4,Cam Newton,34,46,37
3,QB,2011,4,Tarvaris Jackson,31,29,37
4,QB,2011,4,Tony Romo,36,33,36


In [55]:
pos = 'K' # and run 'DST'
key = (pos, '2011', '4')
titles = d[key][3]
players = d[key][0]
stats = d[key][4]

stats[0]
d[key][1]
stats = d[key][4]
stats[0]
key
key_prevs = [tuple([k for k in list(key)[:-1]] + [str(int(key[2])-i)]) for i in range(1, 4)]
d[key][4][0]

['20.00',
 '14.00',
 '14.00',
 '12.00',
 '12.00',
 '12.00',
 '11.00',
 '11.00',
 '11.00',
 '11.00',
 '10.00',
 '10.00',
 '9.00',
 '9.00',
 '8.00',
 '7.00',
 '7.00',
 '7.00',
 '7.00',
 '7.00',
 '6.00',
 '6.00',
 '5.00',
 '5.00',
 '5.00',
 '5.00',
 '5.00',
 '5.00',
 '4.00',
 '4.00',
 '4.00',
 '4.00']

In [57]:
## Data Cleaning for kicker and DST
pos = 'K' # and run 'DST'
key = (pos, '2011', '4')
titles = d[key][3]
players = d[key][0]
stats = d[key][4]

# make a pandas dataframe
df2 = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'team', 'Fantasy Points']
                  + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                  + [t + '_prev_1yr' for t in titles])

for year in range(2011, 2018):
    for week in range(4, 18):
        key = (pos, str(year), str(week))
        # get statistics types, name of players, and statistics
        titles = d[key][3]
        players = d[key][0]
        stats = d[key][4]

        # make a new dataframe 
        df = pd.DataFrame(columns=['pos', 'year', 'week', 'player', 'team', 'Fantasy Points']
                          + [t + '_prev' for t in titles] + [t+'_prev_2wk' for t in titles] + [t+'_prev_3wk' for t in titles]
                          + [t + '_prev_1yr' for t in titles])

        df['player'] = players
        df['pos'] = list(key)[0]
        df['year'] = list(key)[1]
        df['week'] = list(key)[2]
        df['team'] = d[key][1]
        df['Fantasy Points'] = d[key][4][0]

        ## extract data of the previous weeks
        # keys for previous 3 weeks
        key_prevs = [tuple([k for k in list(key)[:-1]] + [str(int(key[2])-i)]) for i in range(1, 4)]

        # add data of previous weeks respectively
        for prev in range(len(key_prevs)):
            # extract key
            key_prev = key_prevs[prev]
            players_prev = d[key_prev][0]
            stats_prev = d[key_prev][4]

            # add all information of the previous week
            for idx_prev in range(len(players_prev)):
                # check if the player exists
                if players_prev[idx_prev] in players:
                    idx = players.index(players_prev[idx_prev])
                else:
                    continue
                for idx_col in range(len(stats_prev)):
                    df.iloc[idx, idx_col+6+len(titles)*prev] = stats_prev[idx_col][idx_prev]

        ## extract data of the previous year
        key_yearly = (pos, str(year-1), 'all')
        players_prev_yearly = d_yearly[key_yearly][0]
        stats_prev_yearly = d_yearly[key_yearly][2][1:]

        # add all information of the previous year
        for idx_prev in range(len(players_prev_yearly)):
            # check if the player exists
            if players_prev_yearly[idx_prev] in players:
                idx = players.index(players_prev_yearly[idx_prev])
            else:
                continue

            for idx_col in range(len(stats_prev_yearly)):
                df.iloc[idx, idx_col+6+len(titles)*len(key_prevs)] = stats_prev_yearly[idx_col][idx_prev]

        df2 = df2.append(df)

for title in titles:
    temp_1wk = title + '_prev'
    temp_2wk = title + '_prev_2wk'
    temp_3wk = title + '_prev_3wk'

    df_temp = df2[[temp_1wk, temp_2wk, temp_3wk]]
    df_temp = df_temp.astype(float)
    df2[title+'_momentum'] = df_temp.apply(lambda r: np.nan if r.isnull().sum() >= 2 else r.mean(), axis=1)

df3 = df2[['pos', 'year', 'week', 'player', 'team', 'Fantasy Points'] + [t + '_prev' for t in titles] 
          + [t + '_momentum' for t in titles] + [t + '_prev_1yr' for t in titles]]
# remove ',' from numbers (e.g.: 1,523 -> 1523)
#df3['Passing Yards_prev_1yr'] = pd.Series(df3['Passing Yards_prev_1yr']).str.replace(',', '');

df3.to_csv(pos+'_adj.csv', index=False)

In [4]:
## Change team names to abbreviation
df = pd.read_csv('DST.csv')
df.head(1)

,pos,year,week,player,Fantasy Points,Fantasy Points_prev,Sacks_prev,Interceptions_prev,Safeties_prev,Fumble Recoveries_prev,...,Sacks_prev_1yr,Interceptions_prev_1yr,Safeties_prev_1yr,Fumble Recoveries_prev_1yr,Blocked Kicks_prev_1yr,Touchdowns_prev_1yr,Points Against_prev_1yr,Passing Yards Allowed_prev_1yr,Rushing Yards Allowed_prev_1yr,Total Yards Allowed_prev_1yr
0,DST,2011,4,Baltimore Ravens,32.0,22.0,5.0,1.0,0.0,1.0,...,27.0,19.0,0.0,8.0,0.0,4.0,270.0,224.9,93.9,318.9


In [58]:
name_info = \
"""
ARI	Arizona Cardinals
ATL	Atlanta Falcons
BAL	Baltimore Ravens
BUF	Buffalo Bills
CAR	Carolina Panthers
CHI	Chicago Bears
CIN	Cincinnati Bengals
CLE	Cleveland Browns
DAL	Dallas Cowboys
DEN	Denver Broncos
DET	Detroit Lions
GB	Green Bay Packers
HOU	Houston Texans
IND	Indianapolis Colts
JAX	Jacksonville Jaguars
KC	Kansas City Chiefs
LAC	Los Angeles Chargers
LAC	San Diego Chargers
LAR	Los Angeles Rams
LAR	St. Louis Rams
MIA	Miami Dolphins
MIN	Minnesota Vikings
NE	New England Patriots
NO	New Orleans Saints
NYG	New York Giants
NYJ	New York Jets
OAK	Oakland Raiders
PHI	Philadelphia Eagles
PIT	Pittsburgh Steelers
SEA	Seattle Seahawks
SF	San Francisco 49ers
TB	Tampa Bay Buccaneers
TEN	Tennessee Titans
WAS	Washington Redskins
"""

In [59]:
name_info2 = name_info.split('\n')[1:-1]

In [60]:
name_info3 = [n.split('\t')[::-1] for n in name_info2]

In [61]:
dict_name_info = dict(name_info3)
dict_name_info

{'Arizona Cardinals': 'ARI',
 'Atlanta Falcons': 'ATL',
 'Baltimore Ravens': 'BAL',
 'Buffalo Bills': 'BUF',
 'Carolina Panthers': 'CAR',
 'Chicago Bears': 'CHI',
 'Cincinnati Bengals': 'CIN',
 'Cleveland Browns': 'CLE',
 'Dallas Cowboys': 'DAL',
 'Denver Broncos': 'DEN',
 'Detroit Lions': 'DET',
 'Green Bay Packers': 'GB',
 'Houston Texans': 'HOU',
 'Indianapolis Colts': 'IND',
 'Jacksonville Jaguars': 'JAX',
 'Kansas City Chiefs': 'KC',
 'Los Angeles Chargers': 'LAC',
 'Los Angeles Rams': 'LAR',
 'Miami Dolphins': 'MIA',
 'Minnesota Vikings': 'MIN',
 'New England Patriots': 'NE',
 'New Orleans Saints': 'NO',
 'New York Giants': 'NYG',
 'New York Jets': 'NYJ',
 'Oakland Raiders': 'OAK',
 'Philadelphia Eagles': 'PHI',
 'Pittsburgh Steelers': 'PIT',
 'San Diego Chargers': 'LAC',
 'San Francisco 49ers': 'SF',
 'Seattle Seahawks': 'SEA',
 'St. Louis Rams': 'LAR',
 'Tampa Bay Buccaneers': 'TB',
 'Tennessee Titans': 'TEN',
 'Washington Redskins': 'WAS'}

In [32]:
df['player'] = df['player'].apply(lambda s: dict_name_info[s])

In [35]:
df.to_csv('DST_adj.csv', index=False)

In [63]:
# remove , from data

df = pd.read_csv('RB_adj2.csv')
df['Rushing Yards_prev_1yr'] = pd.Series(df['Rushing Yards_prev_1yr']).str.replace(',', '');
df.to_csv('RB_adj2.csv', index=False)

In [64]:
df = pd.read_csv('TE_adj2.csv')
df['Receiving Yards_prev_1yr'] = pd.Series(df['Receiving Yards_prev_1yr']).str.replace(',', '');
df.to_csv('TE_adj2.csv', index=False)


In [65]:
df = pd.read_csv('WR_adj2.csv')
df['Receiving Yards_prev_1yr'] = pd.Series(df['Receiving Yards_prev_1yr']).str.replace(',', '');
df.to_csv('WR_adj2.csv', index=False)
